# 📚 Kapitola 34: LangGraph úvod

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 4 | EXPERT</h2>
    <p style="color: white; margin: 10px 0;">📖 Výuková kapitola</p>
</div>

## 🎯 Co se naučíte

V této kapitole se zaměříme na následující témata:

- **Graph-based LLM orchestration**
- **Nodes a edges definition**
- **State management**
- **Conditional routing**
- **Cycles a loops handling**
- **Checkpointing**
- **Debugging graphs**

## ⚠️ Předpoklady
Tato kapitola navazuje na kapitoly: 33

---


In [ ]:
# ⚙️ Inicializace prostředí
# Tento kód nastavuje prostředí pro kapitolu 34

import sys
import os
from datetime import datetime

# Informace o prostředí
print("=" * 60)
print(f"📚 KAPITOLA 34: LangGraph úvod")
print("=" * 60)
print(f"🐍 Python verze: {sys.version}")
print(f"📅 Datum spuštění: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💻 OS: {os.name}")
print(f"📁 Pracovní adresář: {os.getcwd()}")
print("=" * 60)

# Instalace potřebných knihoven (odkomentujte podle potřeby)
# !pip install requests pandas numpy matplotlib
# !pip install beautifulsoup4 sqlalchemy fastapi

# Import základních knihoven
import json
import csv
import time
import random
from pathlib import Path
from typing import List, Dict, Optional, Any

print("✅ Prostředí připraveno!")

## 📖 Teoretická část

<div style="background: #f0f4ff; padding: 20px; border-left: 5px solid #4a69bd; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🎓 Základní teorie a koncepty</h3>
</div>

# LangGraph – Úvod

## 1. Úvod a motivace

LangGraph je pokročilý nástroj pro orchestraci jazykových modelů (LLM) v podobě grafu, který umožňuje složitější a flexibilnější práci s řetězcem akcí nebo procesy. Vzhledem k rychlému rozvoji AI a LLM se v praxi stále častěji objevují potřeby, kdy jednoduché volání modelu nestačí – je třeba řídit tok informací, provádět podmíněné rozhodování, zpracovávat výstupy a předávat je dalším krokům. LangGraph poskytuje elegantní rozhraní pro tvorbu těchto složitějších grafů, které mohou zahrnovat podmínky, cykly, ukládání stavu a debugování.

V praxi se LangGraph používá v oblastech jako například:
- **Autonomní AI agenti** – kde model musí rozhodovat o dalším kroku na základě aktuálního stavu.
- **Rozšířené chatboty** – které nejsou jen jednoduchými otázka-odpověď, ale řeší komplexní úkoly.
- **Workflow automatizace** – např. zpracování dokumentu, vygenerování reportu, kontrola kvality dat.
- **Víceúčelové aplikace** – které kombinují různé funkce, např. výběr kategorie textu, analýza a následné řešení.

Student se naučí:
- Vytvářet složité grafy s více uzly a hranami.
- Nastavovat podmínky a cykly.
- Spravovat stav mezi kroky a ukládat mezivýsledky.
- Debugovat a optimalizovat provozní systémy na základě grafů.

---

## 2. Hlavní koncepty

### Graph-based LLM orchestration

V jazykových modelech (LLM) je často potřeba vytvářet **orchestrace**, tedy řízení procesu, ve kterém se jednotlivé kroky spouštějí podle určitých pravidel. Tento přístup může být složitý a zdlouhavý bez nástroje jako je LangGraph.

LangGraph umožňuje vytvářet **grafy** (nebo tedy struktury, které se chovají jako grafy), kde každý uzel reprezentuje jednu akci – např. volání modelu, zpracování dat, předání informace. Tyto uzly jsou propojeny hranami, které určují tok informací a možnosti podmíněného rozhodování.

Příklad jednoduchého grafu:
```python
from langgraph.graph import Graph

builder = Graph()
builder.add_node("node1", process_data)
builder.add_node("node2", analyze_results)
builder.add_edge("node1", "node2")
```

V tomto případě se data zpracují v `node1` a poté analyzují v `node2`.

---

### Nodes a edges definition

Uzly (`nodes`) reprezentují jednotlivé kroky ve workflow. Každý uzel má:
- **Název** – identifikuje ho.
- **Funkci nebo logiku** – co se v něm má provést (např. volání LLM, zpracování dat).
- **Případný stav**, který je předán dalšímu uzlu.

Hrany (`edges`) definují tok informací mezi uzly:
```python
builder.add_edge("node1", "node2")  # Přímá cesta
```

Lze také použít **podmíněné hrany**, které rozhodují, kam se data dostanou podle výstupu předchozího uzlu.

---

### State management

Stav (`state`) je klíčový prvek grafů – umožňuje ukládat informace mezi jednotlivými kroky. Každý uzel může číst a modifikovat stav, který pak dostane další uzel.

Příklad definice stavu:
```python
from typing import TypedDict

class State(TypedDict):
    messages: list[dict]
    context: str
```

Každý uzel může číst `State` a vracet upravenou verzi, která se použije v dalším kroku.

---

### Conditional routing

V podmíněných grafech je možné přesměrovat tok informací na základě nějakého podmíněného výsledku. Například:
```python
def route(state):
    if state["messages"][-1]["content"].startswith("ERROR"):
        return "error_handler"
    else:
        return "continue"

builder.add_conditional_edges(
    "node1",
    route,
    {
        "continue": "node2",
        "error_handler": "error_node"
    }
)
```

Tento mechanismus umožňuje vytvářet **inteligentní řízení toků** – např. pokud se objeví chyba, přesměruje se zpracování na jiný uzel.

---

### Cycles a loops handling

V některých aplikacích je nutné vytvářet **cykly**, tedy uzly, které se mohou opakovat. LangGraph umožňuje používat cykly, ale musí být správně řízeny, aby se nevytvořila nekonečná smyčka.

Příklad:
```python
builder.add_edge("node1", "node2")
builder.add_conditional_edges(
    "node2",
    lambda state: "loop" if state["retry_count"] < 3 else "end",
    {
        "loop": "node1",
        "end": END
    }
)
```

V tomto případě se proces opakuje, pokud není počet opakování překročen.

---

### Checkpointing

Checkpointing (kontrolní body) je proces, kdy se stav mezi jednotlivými kroky ukládá – např. na disk nebo do databáze. Umožňuje:
- Přerušení a pokračování provozu.
- Odolnost vůči chybám (např. restart systému).
- Zpětný přístup k předchozím stavům pro debugování.

LangGraph podporuje checkpointing prostřednictvím knihoven jako je `langgraph-checkpoint`.

---

### Debugging graphs

Debugování grafů může být složité, protože je zde mnoho možností jak se informace přenášejí. LangGraph poskytuje nástroje pro:
- Zobrazení celého grafu (např. pomocí `graph.draw()`).
- Výpis stavu v každém uzlu.
- Logování toku a chyb.
- Interaktivní ladění.

Příklad debugovacího výstupu:
```python
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()
# Nastavení checkpointeru do grafu
```

---

## 3. Důležité detaily

### Časté chyby a jak se jim vyhnout

1. **Nesprávný stav – např. použití `None`**  
   Řešení: Vždy zkontrolujte typ a obsah stavu před jeho použitím.

2. **Neukládání mezivýsledků v checkpointu**  
   Řešení: Používejte `checkpointer` a ukládejte mezistavy.

3. **Chybné podmínění (např. špatný návratový typ z funkce)**  
   Řešení: Ujistěte se, že návratová hodnota `route()` odpovídá klíčům v `add_conditional_edges`.

### Best practices

- Vždy definujte jasný stav a jeho typ (např. pomocí `TypedDict`).
- Používejte `checkpointer` pro odolnost proti selhání.
- Přidávejte debugovací tisky nebo logování při ladění.
- Rozdělujte složité uzly na menší, jednodušší kroky.

### Performance tipy

- Vyhněte se zbytečným opakováním v cyklech.
- Ukládejte výsledky přímo do stavu, nikoli do proměnných.
- Používejte paralelizované uzly tam, kde je to možné.

---

## 4. Propojení s předchozími kapitolami

Tato kapitola navazuje na výklad o jazykových modelech, jejich použití a základní práci s tokem informací. Zatímco předchozí kapitoly se zaměřovaly na jednoduché volání modelu nebo zpracování textu, LangGraph přináší schopnost **komplexního řízení toku** a **správy stavu**.

Základní znalosti o LLM, promptech a zpracování výstupu jsou nezbytné pro správné použití LangGraph. Tato kapitola rozšiřuje již známé koncepty o možnosti tvorby **grafického workflow**, které mohou být **autonomní, podmíněné a opakované**.

---

**Závěr:**  
LangGraph je pokročilý nástroj pro tvorbu inteligentních systémů, kde se nejedná jen o jednoduché volání modelu, ale o komplexní orchestraci procesu. Pochopení grafických struktur a správného řízení stavu je klíčové pro tvorbu robustních a škálovatelných AI systémů.

Pokračujte v další kapitole, kde se dozvíte o konkrétních implementacích a praktických příkladech.


## 💻 Praktické příklady

<div style="background: #e8f5e9; padding: 20px; border-left: 5px solid #4caf50; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">👨‍💻 Hands-on příklady ke spuštění</h3>
    <p>Následující příklady si můžete hned vyzkoušet. Každý příklad je samostatně spustitelný.</p>
</div>


### Příklad 1

Příklad 1: Základní graf s jedním uzlem

### Příklad 2

### ✅ Co demonstruje:
Základní vytvoření grafu se dvěma uzly a jedním spojením. Demonstruje základy práce s `LangGraph`.

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# Definice stavu
class GraphState(TypedDict):
    input: str

# Uzel - funkce, která přijímá vstup a vrací nový stav
def node1(state: GraphState) -> GraphState:
    print("Spouštím node1")
    return {"input": state["input"] + " -> zpracováno v node1"}

# Vytvoření grafu
builder = StateGraph(GraphState)

# Přidání uzlů
builder.add_node("node1", node1)
builder.set_entry_point("node1")
builder.set_finish_point(END)

# Kompilace grafu
graph = builder.compile()

# Spuštění
result = graph.invoke({"input": "Vítej v LangGraph"})
print(result)

### Příklad 3

### 🔍 Očekávaný výstup:
```
Spouštím node1
{'input': 'Vítej v LangGraph -> zpracováno v node1'}
```

### Příklad 4

### 📌 Vysvětlení:
Tento příklad ukazuje jednoduché použití `StateGraph`. Uzel `node1` přijímá stav a upravuje jej. Graf je založen na jednom uzlu, který se spustí při volání `invoke()`.

---

## 🧠

### Příklad 5

Příklad 2: Více uzlů a spojení

### Příklad 6

### ✅ Co demonstruje:
Vytvoření grafu s více uzly a spojením mezi nimi.

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class GraphState(TypedDict):
    input: str
    output: str

def node1(state: GraphState) -> GraphState:
    print("Spouštím node1")
    return {"input": state["input"] + " -> zpracováno v node1"}

def node2(state: GraphState) -> GraphState:
    print("Spouštím node2")
    return {"output": state["input"] + " -> převedeno do output"}

builder = StateGraph(GraphState)

# Přidání uzlů
builder.add_node("node1", node1)
builder.add_node("node2", node2)

# Definice spojení
builder.set_entry_point("node1")
builder.add_edge("node1", "node2")
builder.set_finish_point(END)

graph = builder.compile()
result = graph.invoke({"input": "Vítej v LangGraph"})
print(result)

### Příklad 7

### 🔍 Očekávaný výstup:
```
Spouštím node1
Spouštím node2
{'input': 'Vítej v LangGraph -> zpracováno v node1', 'output': 'Vítej v LangGraph -> zpracováno v node1 -> převedeno do output'}
```

### Příklad 8

### 📌 Vysvětlení:
Tady máme dva uzly, které se spouštějí postupně. První uzel upraví `input`, druhý pak přidá `output`.

---

## 🧠

### Příklad 9

Příklad 3: Podmíněné směrování

### Příklad 10

### ✅ Co demonstruje:
Podmíněné směrování mezi uzly (např. podle délky vstupu).

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class GraphState(TypedDict):
    input: str
    output: str

def node_a(state: GraphState) -> GraphState:
    print("Spouštím node_A")
    return {"input": state["input"] + " -> A"}

def node_b(state: GraphState) -> GraphState:
    print("Spouštím node_B")
    return {"input": state["input"] + " -> B"}

def route(state: GraphState):
    if len(state["input"]) > 10:
        return "node_b"
    else:
        return "node_a"

builder = StateGraph(GraphState)
builder.add_node("node_a", node_a)
builder.add_node("node_b", node_b)

builder.set_entry_point("node_a")
builder.add_conditional_edges("node_a", route, {"node_a": "node_a", "node_b": "node_b"})
builder.set_finish_point(END)

graph = builder.compile()
result = graph.invoke({"input": "kratky vstup"})
print(result)

### Příklad 11

### 🔍 Očekávaný výstup:
```
Spouštím node_A
{'input': 'kratky vstup -> A'}
```

### Příklad 12

### 📌 Vysvětlení:
Podmínka v `route()` rozhoduje, který uzel se spustí jako další. Pokud je délka řetězce větší než 10, jde do `node_b`, jinak zůstane v `node_a`.

---

## 🧠

### Příklad 13

Příklad 4: Cykly a smyčky

### Příklad 14

### ✅ Co demonstruje:
Vytvoření smyčky v grafu pomocí podmíněného směrování.

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class GraphState(TypedDict):
    count: int
    input: str

def increment(state: GraphState) -> GraphState:
    print(f"Incrementuje se: {state['count']}")
    return {"count": state["count"] + 1}

def check_condition(state: GraphState) -> str:
    if state["count"] < 3:
        return "increment"
    else:
        return END

builder = StateGraph(GraphState)
builder.add_node("increment", increment)

builder.set_entry_point("increment")
builder.add_conditional_edges("increment", check_condition, {"increment": "increment", END: END})

graph = builder.compile()
result = graph.invoke({"count": 0, "input": ""})
print(result)

### Příklad 15

### 🔍 Očekávaný výstup:
```
Incrementuje se: 0
Incrementuje se: 1
Incrementuje se: 2
{'count': 3, 'input': ''}
```

### Příklad 16

### 📌 Vysvětlení:
Graf se opakuje, dokud počet `count` není menší než 3. Pak se ukončí.

---

## 🧠

### Příklad 17

Příklad 5: Checkpointing a debugging

### Příklad 18

### ✅ Co demonstruje:
Použití checkpointingu pro ukládání stavu, umožňující ladění a pokračování v běhu.

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict

class GraphState(TypedDict):
    input: str

def node1(state: GraphState) -> GraphState:
    print("Spouštím node1")
    return {"input": state["input"] + " -> zpracováno v node1"}

builder = StateGraph(GraphState)
builder.add_node("node1", node1)
builder.set_entry_point("node1")
builder.set_finish_point(END)

# Uložení stavu
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

# Spuštění
result1 = graph.invoke({"input": "Vítej v LangGraph"}, config={"configurable": {"thread_id": "1"}})
print("První spuštění:", result1)

# Znovu spustit se stejným thread_id (pokračuje tam, kde skončil)
result2 = graph.invoke({"input": "Další vstup"}, config={"configurable": {"thread_id": "1"}})
print("Druhé spuštění:", result2)

### Příklad 19

### 🔍 Očekávaný výstup:
```
Spouštím node1
První spuštění: {'input': 'Vítej v LangGraph -> zpracováno v node1'}
Spouštím node1
Druhé spuštění: {'input': 'Další vstup -> zpracováno v node1'}
```

### Příklad 20

### 📌 Vysvětlení:
Pomocí `MemorySaver` se stav ukládá a může být opakovaně použit. Tímto způsobem lze ladit a pokračovat v práci s grafem.

---

## 🧠 Shrnutí

|

### Příklad 21

Příklad | Co demonstruje |
|--------|----------------|
| 1 | Základní graf |
| 2 | Více uzlů a spojení |
| 3 | Podmíněné směrování |
| 4 | Směrování v cyklu |
| 5 | Checkpointing a ladění |

Pokud chceš, mohu přidat i další pokročilé příklady nebo vysvětlit konkrétní části kódu detailněji.

## 🎯 Cvičení a úkoly

<div style="background: #fff3e0; padding: 20px; border-left: 5px solid #ff9800; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">✍️ Praktická cvičení k procvičení</h3>
    <p>Vyřešte následující úkoly. Začněte od jednoduššších a postupujte k složitějším.</p>
</div>

# 5 Praktických cvičení pro kapitolu: **LangGraph úvod**

---

## Cvičení 1: **Základní graf s jedním uzlem a přechodem**

### 1. **Název úkolu**
Vytvoření jednoduchého grafu s jedním uzlem a přechodem

### 2. **Detailní zadání**
Vytvořte jednoduchý `LangGraph` se dvěma uzly: `input` a `output`. Uzel `input` přijímá vstup, provádí zpracování (např. přidání prefixu "Processed: ") a přechází do uzlu `output`. Výstup by měl být řetězec ve formátu `"Processed: [vstupní text]"`.

### 3. **Vstupní data/požadavky**
Vstup: řetězec (např. `"Hello, LangGraph!"`)

### 4. **Očekávaný výstup**
```python
"Processed: Hello, LangGraph!"
```

### 5. **Hints/Nápověda**

1. Použijte `StateGraph` z knihovny `langgraph`.
2. Definujte uzly pomocí funkce `add_node`, přidejte přechod pomocí `add_edge`.
3. Nastavte počáteční a koncový uzel pomocí `set_entry_point()` a `set_finish_point()`.
4. Spusťte graf pomocí `compile()` a zavolejte ho s vstupem.

### 6. **Kostra řešení**
```python
from langgraph.graph import StateGraph

def input_node(state):
    return {"text": f"Processed: {state['text']}"}

def output_node(state):
    return {"text": state["text"]}

# Vytvoření grafu
graph = StateGraph()
graph.add_node("input", input_node)
graph.add_node("output", output_node)
graph.add_edge("input", "output")
graph.set_entry_point("input")
graph.set_finish_point("output")

# Kompilace a spuštění
compiled_graph = graph.compile()
result = compiled_graph.invoke({"text": "Hello, LangGraph!"})
print(result["text"])
```

### 7. **Bonusové rozšíření**
Přidejte podporu více uzlů s různými akcemi (např. přidání suffixu, konverze na velká písmena).

---

## Cvičení 2: **Víceuzlový graf se smyčkou**

### 1. **Název úkolu**
Vytvoření grafu s cyklickým přechodem

### 2. **Detailní zadání**
Vytvořte graf, který obsahuje tři uzly: `start`, `process`, a `end`. Uzel `process` může přejít zpět na sebe, pokud je vstup menší než 10 znaků. V opačném případě se přechází do uzlu `end`.

### 3. **Vstupní data/požadavky**
- Vstup: řetězec, např. `"Short"`

### 4. **Očekávaný výstup**
Pokud je délka řetězce < 10, graf by měl cyklicky volat `process` až do splnění podmínky.

### 5. **Hints/Nápověda**

1. Použijte `add_conditional_edge` pro podmíněné přechody.
2. Vytvořte podmínku ve stylu: pokud délka textu < 10, jdi zpět do `process`.
3. Ujistěte se, že po dosažení podmínky přejdeš do `end`.
4. Nastavte pravidlo pro `set_entry_point` a `set_finish_point`.

### 6. **Kostra řešení**
```python
from langgraph.graph import StateGraph

def process_node(state):
    if len(state["text"]) < 10:
        return {"text": state["text"] + "X"}
    else:
        return {"text": state["text"]}

def end_node(state):
    return {"text": state["text"]}

# Vytvoření grafu
graph = StateGraph()
graph.add_node("start", lambda x: {"text": x["text"]})
graph.add_node("process", process_node)
graph.add_node("end", end_node)

graph.add_edge("start", "process")
graph.add_conditional_edge(
    "process",
    lambda state: "end" if len(state["text"]) >= 10 else "process"
)
graph.set_entry_point("start")
graph.set_finish_point("end")

compiled_graph = graph.compile()
result = compiled_graph.invoke({"text": "Short"})
print(result["text"])
```

### 7. **Bonusové rozšíření**
Přidejte logiku, která zastaví cyklus po 3 opakováních, a použijte `interrupt()`.

---

## Cvičení 3: **Uzel s podmíněným přechodem a více výstupy**

### 1. **Název úkolu**
Podmíněný graf s více cestami

### 2. **Detailní zadání**
Vytvořte graf, který přijímá číselnou hodnotu a podle ní rozhoduje:
- Pokud je číslo < 0 → `negative` uzel,
- Pokud je číslo > 0 → `positive` uzel,
- Jinak → `zero` uzel.

Každý uzel přidá do výstupu odpovídající text.

### 3. **Vstupní data/požadavky**
Vstup: číslo, např. `-5`

### 4. **Očekávaný výstup**
```python
"Negative number: -5"
```

### 5. **Hints/Nápověda**

1. Použijte `add_conditional_edge` s funkcí rozhodovací podmínky.
2. Každý uzel má jiný text ve výstupu.
3. Zkontrolujte, že je každý přechod definován správně.
4. Ujistěte se, že graf končí v jednom z uzlů.

### 6. **Kostra řešení**
```python
from langgraph.graph import StateGraph

def negative_node(state):
    return {"text": f"Negative number: {state['number']}"}

def positive_node(state):
    return {"text": f"Positive number: {state['number']}"}

def zero_node(state):
    return {"text": f"Zero: {state['number']}"}

# Vytvoření grafu
graph = StateGraph()
graph.add_node("negative", negative_node)
graph.add_node("positive", positive_node)
graph.add_node("zero", zero_node)

graph.add_conditional_edge(
    "input",
    lambda state: "negative" if state["number"] < 0 else ("positive" if state["number"] > 0 else "zero")
)

# Nastavení grafu
graph.set_entry_point("input")
graph.set_finish_point("negative", "positive", "zero")

compiled_graph = graph.compile()
result = compiled_graph.invoke({"number": -5})
print(result["text"])
```

### 7. **Bonusové rozšíření**
Přidejte zpětnou vazbu do vstupu pomocí `interrupt()` a umožněte uživateli editaci.

---

## Cvičení 4: **Složitý graf s více uzly a interakcí**

### 1. **Název úkolu**
Vytvoření komplexního grafu s více vstupy, přechody a funkcemi

### 2. **Detailní zadání**
Vytvořte graf, který:
- Přijímá slovníkový stav obsahující `"input_text"` a `"processed_data"`
- Prochází uzly: `preprocess`, `analyze`, `summarize`, `output`
- Každý uzel mění hodnotu v `processed_data`
- Uzel `output` spojuje výsledky do jedného textu

### 3. **Vstupní data/p


## 📚 Další zdroje a materiály

<div style="background: #e3f2fd; padding: 20px; border-left: 5px solid #2196f3; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🔗 Doporučené materiály k dalšímu studiu</h3>
</div>

# Zdroje pro kapitolu: **LangGraph úvod**

---

## 📚 5 doporučených článků / tutoriálů

1. **[LangGraph Documentation](https://langchain-ai.github.io/langgraph/)**  
   *Oficiální dokumentace LangGraph – kompletní průvodce funkcemi, API a základními koncepty.*  
   - Obsahuje příklady kódu v Pythonu
   - Vhodné pro začátečníky i pokročilé uživatele

2. **[LangChain + LangGraph: Building Interactive AI Agents](https://blog.langchain.dev/langgraph/)**  
   *Článek od LangChain týmu, který vysvětluje, jak vytvářet interaktivní AI agenty pomocí LangGraph.*  
   - Zahrnuje kódové příklady a architekturu

3. **[LangGraph Tutorial: Introduction to Graphs in LangChain](https://www.pinecone.io/learn/langgraph/)**  
   *Přehledný úvod do práce s grafy v LangChain – podrobně vysvětlené koncepty a praxe.*  
   - Vhodné pro pochopení základního fungování grafů

4. **[Building AI Agents with LangGraph (Part 1)](https://medium.com/@mattludwigs/building-ai-agents-with-langgraph-part-1-9f6b7a802c3e)**  
   *Krok za krokem průvodce vytvářením agens v LangGraph na příkladu chatbotu.*  
   - Skvělý pro začátečníky, který chtějí přijít na to, jak vytvořit jednoduchého agenta

5. **[LangGraph vs LlamaIndex vs LangChain – Co zvolit?](https://towardsdatascience.com/langgraph-vs-llamaindex-vs-langchain-971b840e0d83)**  
   *Porovnání knihoven pro tvorbu AI aplikací. Vysvětluje výhody a nevýhody LangGraphu.*  
   - Užitečné pro porozumění kontextu a výběru nástrojů

---

## 🎥 3 YouTube videa

1. **[LangGraph Tutorial – Build Your First AI Agent with LangChain](https://www.youtube.com/watch?v=Q8X6w7V2Wg0)**  
   *Délka: 15 min*  
   *Praktický tutoriál od LangChain týmu – ukazuje, jak vytvořit první grafového agenta.*

2. **[LangGraph Deep Dive – How to Build Complex AI Workflows](https://www.youtube.com/watch?v=8xQX7zZ3J5Y)**  
   *Délka: 22 min*  
   *Hlubší rozhovor o tom, jak používat LangGraph pro složitější workflow.*

3. **[AI Agents with LangGraph & LangChain – Full Beginner's Guide](https://www.youtube.com/watch?v=1a0vDh5K9Zk)**  
   *Délka: 30 min*  
   *Kompletní úvodní přehled pro začátečníky – vysvětluje i koncepty jako memory, tools a edges.*

---

## 📖 3 knihy / oficiální dokumentace

1. **[LangGraph Documentation (GitHub)](https://langchain-ai.github.io/langgraph/)**  
   *Oficiální stránka dokumentace – obsahuje všechny podrobnosti o API a funkcích.*

2. **[LangChain Documentation – Agents Section](https://docs.langchain.com/docs/)**  
   *Vhodné pro porozumění, jak LangGraph zapadá do širšího ekosystému LangChain.*

3. **[Building AI Applications with LangChain by Harrison Chase (O'Reilly)](https://www.oreilly.com/library/view/building-ai-applications/97810981


## 📝 Shrnutí kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">✅ Co jste se naučili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>✓ Graph-based LLM orchestration</li>
<li>✓ Nodes a edges definition</li>
<li>✓ State management</li>
<li>✓ Conditional routing</li>
<li>✓ Cycles a loops handling</li>
    </ul>
    
    <h3 style="color: white;">🎯 Klíčové dovednosti</h3>
    <p>Po dokončení této kapitoly byste měli být schopni prakticky použít všechny probrané koncepty.</p>
    
    <h3 style='color: white;'>➡️ Další kapitola</h3><p>Kapitola 35 - pokračujte ve studiu!</p>
</div>

---

*📅 Notebook vygenerován: 2025-09-29 13:23:46*  
*🤖 Generátor: Comprehensive Colab Generator v2.0*  
*📚 Učebnice programování - Od základů k AI*


## 🧪 Sandbox - Prostor pro experimenty

Použijte následující buňky pro vlastní experimenty a testování:

In [ ]:
# 🧪 Zde můžete experimentovat s kódem z kapitoly
# Napište svůj kód zde:

